In [7]:
import os
import shutil
import glob
import json
import time
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from maxio.tools.rM2svg import rm2svg
from PyPDF2 import PdfFileReader

## Set this up to work with the test example

In [8]:
refNrPath = "paper"
origPDF = "paper.pdf"
meta = json.loads(open(refNrPath + ".metadata").read())
fname = meta["visibleName"]
input1 = PdfFileReader(open(origPDF, "rb"))
npages = input1.getNumPages()
pdfsize = input1.getPage(0).mediaBox
pdfx = int(pdfsize[2])
pdfy = int(pdfsize[3])

Empty pages don't generate \*.rm files. So we just use a placeholder empty file to simulate it. 

In [9]:
#os.mkdir("tempDir")
emptyRm = "empty.rm"
rm2svg(emptyRm, "tempDir/emptyrm.svg", coloured_annotations=False, x_width=pdfx, y_width=pdfy)

## 1.+2. Loop through the number of pages, creating an annotation pdf for each.

In [10]:
pdflist = []
for pg in range(0, npages):
    rmpath = refNrPath+"/"+str(pg)+".rm"
    if os.path.isfile(rmpath):
        rm2svg(rmpath, "tempDir/temprm" + str(pg) + ".svg", coloured_annotations=True, x_width=pdfx, y_width=pdfy)
        svg_path = "tempDir/temprm" + str(pg) + ".svg"
    else:
        svg_path = "tempDir/emptyrm.svg"
    convertSvg2PdfCmd = "".join(["rsvg-convert -f pdf -o ", "tempDir/temppdf" + str(pg), ".pdf ", svg_path])
    os.system(convertSvg2PdfCmd)
    pdflist.append("tempDir/temppdf"+str(pg)+".pdf")

## 3. merge all pdf pages into one

In [11]:
merged_rm = "tempDir/merged_rm.pdf"
os.system("convert "+ (" ").join(pdflist)+" "+merged_rm)

0

## 4. overlay on top of the existing page

In [12]:
stampCmd = "".join(["pdftk ", origPDF, " multistamp ", merged_rm, " output ", origPDF[:-4], "_annot.pdf"])
os.system(stampCmd)

0